#### STT 테스트

In [19]:
import requests
import json
kakao_speech_url = "https://kakaoi-newtone-openapi.kakao.com/v1/recognize"

rest_api_key = '954052c06ac962dca9151a748fdd9289'

headers = {
    "Content-Type": "application/octet-stream",
    "X-DSS-Service": "DICTATION",
    "Authorization": "KakaoAK " + rest_api_key,
}

with open('heykakao.wav', 'rb') as fp:
    audio = fp.read()

res = requests.post(kakao_speech_url, headers=headers, data=audio)
print(res.text)

------newtoneguXZzo8uCcwUCjK5
Content-Type: application/json; charset=UTF-8

{"type":"beginPointDetection","value":"BPD"}
------newtoneguXZzo8uCcwUCjK5
Content-Type: application/json; charset=UTF-8

{"type":"partialResult","value":"헤이"}
------newtoneguXZzo8uCcwUCjK5
Content-Type: application/json; charset=UTF-8

{"type":"partialResult","value":"헤이 카카오"}
------newtoneguXZzo8uCcwUCjK5
Content-Type: application/json; charset=UTF-8

{"type":"endPointDetection","value":"EPD"}
------newtoneguXZzo8uCcwUCjK5
Content-Type: application/json; charset=UTF-8
Speech-Length: 2

{"type":"finalResult","value":"헤이 카카오","nBest":[{"value":"헤이 카카오","score":99}],"voiceProfile":{"registered":false,"authenticated":false},"durationMS":2880,"qmarkScore":0,"gender":0}
------newtoneguXZzo8uCcwUCjK5--



#### finalResult 추출 방법

In [13]:
result_json_string = res.text[res.text.index('{"type":"finalResult"'):res.text.rindex('}')+1]
result = json.loads(result_json_string)
print(result)
print(result['value'])

{'type': 'finalResult', 'value': '헤이 카카오', 'nBest': [{'value': '헤이 카카오', 'score': 99}], 'voiceProfile': {'registered': False, 'authenticated': False}, 'durationMS': 2880, 'qmarkScore': 0, 'gender': 0}
헤이 카카오


#### TTS 테스트

In [22]:
import os, requests, json, wave
url = "https://kakaoi-newtone-openapi.kakao.com/v1/synthesize"
key = '954052c06ac962dca9151a748fdd9289'
headers = {
    "Content-Type": "application/xml",
    "Authorization": "KakaoAK " + key,
}

data = "<speak>카카오 테스트중 입니다.</speak>".encode('utf-8').decode('latin1')
res = requests.post(url, headers=headers, data=data)
f= open('temp.wav', 'wb')
f.write(res.content)
f.close()

In [27]:
!pip install sounddevice
!pip install soundfile

#### 음성 녹음 test

In [32]:
import time
import queue, os, threading
import sounddevice as sd
import soundfile as sf
from scipy.io.wavfile import write

q = queue.Queue()
recorder = False
recording = False

def complicated_record():
    with sf.SoundFile("test.wav", mode='w', samplerate=16000, subtype='PCM_16', channels=1) as file:
        with sd.InputStream(samplerate=16000, dtype='int16', channels=1, callback=complicated_save):
            while recording:
                file.write(q.get())
        
def complicated_save(indata, frames, time, status):
    q.put(indata.copy())
    
def start():
    global recorder
    global recording
    recording = True
    recorder = threading.Thread(target=complicated_record)
    print('start recording')
    recorder.start()
    
def stop():
    global recorder
    global recording
    recording = False
    recorder.join()
    print('stop recording')
    
start()
time.sleep(3)
stop()

start recording
stop recording
